In [1]:
from esfMRI import sliceWindows, align
from nilearn import connectome
import numpy as np
import pickle
import math
import os

In [2]:
# 可调节参数
window_length_Second = [60, 100, 120, 150, 180]
sliding_steps = [3, 6, 10]

In [3]:
with open("time_series2.pkl", "rb") as f:
    data = pickle.load(f)

In [4]:
save_dir = "dFC"
os.makedirs(save_dir, exist_ok=True)
for time in window_length_Second:
    for step in sliding_steps:
        slidingWindows = {}
        dFCs = {}
        for subid in data:
            slidingWindows[subid] = {}
            dFCs[subid] = {}
            windows_preop = []
            windows_postop = []
            window_length_preop = []
            window_length_postop = []
            slidingWindows[subid]["ses-preop"] = {}
            dFCs[subid]["ses-preop"] = {}
            for run, items in data[subid]["ses-preop"].items():
                stepFrame = math.ceil(step/items["TR"])
                preopFrame = math.ceil(time/items["TR"])
                tmp = sliceWindows(items["time_series"], preopFrame, stepFrame)
                slidingWindows[subid]["ses-preop"][run] = tmp
                dFCs[subid]["ses-preop"][run] = connectome.ConnectivityMeasure(kind="correlation").fit_transform(tmp)
                # 对角线置0
                np.fill_diagonal(dFCs[subid]["ses-preop"][run])
                window_length_preop.append(len(tmp))
                windows_preop += tmp
            slidingWindows[subid]["ses-postop"] = {}
            dFCs[subid]["ses-postop"] = {}
            for run, items in data[subid]["ses-postop"].items():
                stepFrame = math.ceil(step/items["TR"])
                postopFrame = math.ceil(time/items["TR"])
                tmp = sliceWindows(items["time_series"], postopFrame, stepFrame)
                slidingWindows[subid]["ses-postop"][run] = tmp
                dFCs[subid]["ses-postop"][run] = connectome.ConnectivityMeasure(kind="correlation").fit_transform(tmp)
                # 对角线置0
                np.fill_diagonal(dFCs[subid]["ses-postop"][run])
                window_length_postop.append(len(tmp))
                windows_postop += tmp
            slidingWindows[subid]["ses-preop"]["total"] = windows_preop
            slidingWindows[subid]["ses-postop"]["total"] = windows_postop
            slidingWindows[subid]["length"] = [window_length_preop, window_length_postop]
            dFCs[subid]["length"] = [window_length_preop, window_length_postop]
        # 同时保存滑动窗口和动态功能连接
        with open(f"{save_dir}/{time}_{step}_slidingWindows.pkl", "wb") as f:
            pickle.dump(slidingWindows, f)
        with open(f"{save_dir}/{time}_{step}dFCs.pkl", "wb") as f:
            pickle.dump(dFCs, f)